# Basic example using the MOGPTK

One fundamental pillar of MOGPTK are the data and model classes, where one stores the data for each channel and the other defines the model to use. In this tutorial we will show how to instanciate them and use them for a toy dataset.

This tutorial show examples for: __MOSM, CONV, CSM__ and __SM-LMC__ kernels


[Documentation](https://games-uchile.github.io/MultiOutputGP-Toolkit/)

In [1]:
# import library if it is not installed
import sys
sys.path.insert(0, '../')

import mogptk

import numpy as np
import matplotlib.pyplot as plt

# TODO: remove when done:
%reload_ext autoreload
%autoreload 2

## 1. Generate dataset

As a toy example we generate three sinusoidal signals. The four sinusses are each put in a channel and have the same frequency but differ in phase and delay. The observations are randomly picked in the range of [0,4], where the first channel will have the second half of the observations removed. The second channel will be a phased-delayed version, the third will have a second sinusoidal, and the fourth will have a time-delay.

In [2]:
# create data points

# time array
n_points = 100
t = np.linspace(0, 6, n_points)

# data for channel 1
y1 = np.sin(6 * t)
# add noise
y1 += np.random.normal(scale=0.1, size=len(t))

# phased version
y2 = np.sin(6 * t + 2)
y2 += np.random.normal(scale=0.1, size=len(t))

# added sinosoidal
y3 = np.sin(6 * t) - np.sin(4 * t)
y3 += np.random.normal(scale=0.1, size=len(t))

# delayed and amplified
y4 = 3 * np.sin(6 * (t - 2))
y4 += np.random.normal(scale=0.1, size=len(t))

Then for each time series (channels) we will create a data class object, and then we will create a list of data objects.

In [3]:
data_list = []

# data object
data = mogptk.Data(t, y1, name='A')
data_list.append(data)

data = mogptk.Data(t, y2, name='B')
data_list.append(data)

data = mogptk.Data(t, y3, name='C')
data_list.append(data)

data = mogptk.Data(t, y4, name='D')
data_list.append(data)

Then we can remove the seconf half for the channel 'A' and randomly remove 40% for each channel

In [4]:
# remove randomly
for data in data_list:
    data.remove_randomly(pct=0.3)

# remove for channel 0
data_list[0].remove_range(start=2.0, end=None)

We can plot the signal in time and frequency domain

In [ ]:
# plot data in time
for data in data_list:
    data.plot()

In [ ]:
# plot spectrum
for data in data_list:
    data.plot_spectrum()

With the data objects we can use any of the models of the toolkit

## 2. Models

To use a model we first create the object, then we can initialize the parameters and train/predict with ease

When we create the object, we have to pass the data list and the number of components. Initially the kernel parameters are at random, but we can __optionally__ re-initialize them by:

1. 'BNSE': Estimate the PSD for each channel via BNSE (bayesian non parametric spectral estimation, Tobar 2019), then utilize the position, magnitude and with of the estimated spectrum as spectral means, weights and variance respectively.

2. 'SM': For each channel adjust a single output Gaussian process with spectral mixture kernel, and use said parameters as initial parameters for the kernel.

In [ ]:
# create model

model = mogptk.MOSM(data_list, Q=2)
# model = mogptk.CSM(data_list, Q=2)
# model = mogptk.SMLMC(data_list, Q=2)
# model = mogptk.CG(data_list, Q=2)

# initialize parameters of kernel
model.init_params('BNSE')

With the model initialized we can obtain a preliminary prediction __before__ training

In [ ]:
# construct the gpflow model
model.build()

# plot the prediction with untrained model
_ = mogptk.plot_prediction(model)
plt.suptitle('Untrained model', y=1.03, fontsize=20);

### 2.1 Trainning
The model can be trained using tensorflow of scipy optimizers

In [ ]:
model.train(
    method='L-BFGS-B',
    tol=1e-10,
    maxiter=2000)

In [ ]:
_ = mogptk.plot_prediction(model)
plt.suptitle('Trained model', y=1.03, fontsize=20);

## 3. Prediction outside training range

Prediction can be made outside the training range easily using the `model.predict()` function.

For this we construct a list of ndarrays with the prediction inputs. The `model.predict()` returns the posterior mean, upper and lower confidence intervals. 

Upper and lower confidence intervals are used instead of the standar deviation of the posterior because the toolkit admits tranformations of the dataset (see section 4) and if the transformation is not linear -for example a log transformation- the process in the original domain will not be Gaussian.

In [ ]:
# we use the same time for all the channels
input_pred = np.linspace(6, 10, 100)

x_pred = [input_pred for i in range(model.get_output_dims())]

# mean, lower and higher confidence interval
y_pred, y_lo_ci, y_hi_ci = model.predict(x_pred)

The output of the function follows the same structure as the input, a list of length equal to the number of channels, where each element is a numpy array with the predictions on said channel.

This way we can use matplotlib to add the prediction to the one generated in with the training range.

**add confidence intervals**

In [ ]:
# base figure
fig, axarr, data_dict = mogptk.plot_prediction(model)

# add prediction
for i in range(model.get_output_dims()):
    axarr[i].plot(x_pred[i], y_pred[i], '--k')
    axarr[i].fill_between(x_pred[i],
                          y_lo_ci[i],
                          y_hi_ci[i],
                          color='k',
                          alpha=0.25)
                          
plt.suptitle('Prediction outside training range', y=1.03, fontsize=20);

## 4. Transformations on data

Currently the toolkit admits the following transformation on channels:

* Normalization.
* Polinomial detrending.
* Logarithmical.

These can be applied in succesion, the toolkit keeps tracks of them to apply the inverse tranformation on the process when doing predictions.

As en example, lets take the exponential of the original data, for the first 3 channels, and fit the model with log transformation for those channels.

In [ ]:
data_list2 = []

data_list2.append(mogptk.Data(t, np.exp(y1)))
data_list2.append(mogptk.Data(t, np.exp(y2)))
data_list2.append(mogptk.Data(t, np.exp(y3)))
data_list2.append(mogptk.Data(t, y4))

for data in data_list2:
    data.remove_randomly(pct=0.3)

data_list2[0].remove_range(start=2.0, end=None)

In [ ]:
# apply transformations
for data in data_list2[:-1]:
    data.transform(mogptk.TransformLog)
    data.plot()

In [ ]:
model2 = mogptk.MOSM(data_list2, Q=2)
model2.init_params('BNSE')

model2.build()
model2.train()

In [ ]:
_ = mogptk.plot_prediction(model2)
plt.suptitle('Training with Log transformation', y=1.03, fontsize=20);

## 5. Load DataFrames

The toolkit also allows to load datasets directly from a csv file with `mogptk.LoadCSV()`, where we specify the file, and the name of the columns for input and output and it returns a Data class object.

As an example, let's load the price of Oil and gold.

In [ ]:
oil = mogptk.LoadCSV(filename='data/gonu/brent-daily.csv',
                     x_cols='Date',
                     y_col='Price',
                     name='Oil',
                     format={'Date': mogptk.FormatDate})
gold = mogptk.LoadCSV(filename='data/gonu/lmba-gold-usd-am-daily.csv',
                      x_cols='Date',
                      y_col='Price',
                      name='Gold',
                      format={'Date': mogptk.FormatDate})

data_list3 = [oil, gold]

Once the data is loaded, the toolkit have a series of tools to:_ filter, agregate by days, remove ranges and set prediction ranges_ by specifying the date range rather than the internal values used as inputs.

In [ ]:
for channel in data_list3:
    # filter by date
    channel.filter('2015-01-01', '2018-12-31')
    # agregate from daily to weekly
    channel.aggregate('7d')
    
    channel.transform(mogptk.TransformLog)
    channel.transform(mogptk.TransformDetrend)
    
    # remove by date range
    channel.remove_range('2016-11-15', '2017-01-01')
    channel.remove_randomly(pct=0.5)
    
    # set prediction range by date
    channel.set_pred_range('2015-01-01', '2018-12-31', step='1d')
    
    channel.plot()

With this we can train the model normaly and set a prediction range by date

In [ ]:
model3 = mogptk.MOSM(data_list3, Q=3)

model3.init_params('BNSE')

model3.build()
model3.train(method='L-BFGS-B', maxiter=2000, tol=1e-50)

We can set the prediction range by dates and then predict, in this case we call the predict function without arguments as the range where already set

In [ ]:
for i in range(model3.get_output_dims()):    
    model3.data[i].set_pred_range('2019-01-01', '2019-05-01', step='1d')

y_pred, low_ci, hi_ci = model3.predict()
x_pred = model3.get_x_pred()

In [ ]:
# base figure
fig, axarr, data_dict = mogptk.plot_prediction(model3, figsize=(12, 4))

# add outside training range prediction
for i in range(model3.get_output_dims()):
    axarr[i].plot(x_pred[i], y_pred[i], '--k')
    axarr[i].fill_between(x_pred[i].reshape(-1),
                          low_ci[i],
                          hi_ci[i],
                          color='k',
                          alpha=0.25)

plt.suptitle('Oil and Gold dataset', y=1.03, fontsize=20);